In [2]:
import os
import sys
from dotenv import load_dotenv
from elasticsearch import Elasticsearch
from agent_tools import initialize_tools, search_publications

# Load environment
load_dotenv(dotenv_path=".env", override=True)

# Initialize Elasticsearch client (6.x version)
es = Elasticsearch(
    hosts=[os.getenv('ES_HOST')],
    http_auth=(os.getenv('ES_USER'), os.getenv('ES_PASS')),
    verify_certs=False
)

print(f"Connected to ES: {es.ping()}")

# Initialize the tools
initialize_tools(es)
print("Tools initialized!")

Connected to ES: True
Tools initialized!


/Users/filipberntsson/Dev/es_workspace/venv/lib/python3.13/site-packages/elasticsearch/connection/http_urllib3.py:176: UserWarning: Connecting to elk-test.cthb.se using SSL with verify_certs=False is insecure.
  warnings.warn(


In [3]:
# Run a test search
summary = search_publications(
    query="machine learning", 
    filters={"year_range": {"gte": 2022}}, 
    size=10
)

# Display results nicely
print(f"Session ID: {summary['session_id']}")
print(f"Total results: {summary['total_results']}")
print(f"\nFirst few results:")
for i, result in enumerate(summary['sample_results'][:3]):
    print(f"\n{i+1}. {result['title']} ({result['year']})")
    if result['authors']:
        print(f"   Authors: {', '.join(result['authors'][:3])}")
    if result['abstract']:
        print(f"   Abstract: {result['abstract'][:100]}...")

# Show aggregations
print(f"\nYear distribution:")
for year_data in summary['aggregations'].get('years', [])[:5]:
    print(f"  {year_data['value']}: {year_data['count']} papers")

Session ID: 7d826697-17fc-4a78-a301-ad0a4d6cfde3
Total results: 1547

First few results:

1. On uncertainty estimation in machine learning (2024)
   Authors: Jakob Lindqvist
   Abstract: This thesis explores the representation of probabilistic machine learning models,<br />focusing on r...

2. Modelling Cryptographic Distinguishers Using Machine Learning (2022)
   Authors: Carlo Brunetta, Pablo Picazo-Sanchez
   Abstract: Cryptanalysis is the development and study of attacks against cryptographic primitives and protocols...

3. Operationalizing Machine Learning Using Requirements-Grounded MLOps (2024)
   Authors: Milos Bastajic, Jonatan Boman Karinen, Jennifer Horkoff
   Abstract: [Context &amp; Motivation] Machine learning (ML) use has increased significantly, [Question/Problem]...

Year distribution:
  2025: 224 papers
  2024: 489 papers
  2023: 444 papers
  2022: 390 papers
